In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import os
import sys
import random
import seaborn as sns
from matplotlib.font_manager import FontProperties
font1 = FontProperties(fname=r"C:\\Windows\\Fonts\\STZHONGS.ttf", size=14)
font2 = FontProperties(fname=r"C:\\Windows\\Fonts\\STZHONGS.ttf", size=12)
font3 = FontProperties(fname=r"C:\\Windows\\Fonts\\STZHONGS.ttf", size=10)
font4 = FontProperties(fname=r"C:\\Windows\\Fonts\\STZHONGS.ttf", size=7)
sns.set_style('whitegrid')
sns.set_palette("muted")
random.seed(20241120)
np.random.seed(20241120)
torch.manual_seed(3407) # Torch.manual_seed(3407) is all you need. Paper: http://arxiv.org/abs/2109.08203
# 修改工作路径，使本.ipynb文件能够像在本文件夹根目录下一样导入其他模块
# Modify the working path so that this.ipynb file can import other modules like in the root directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(os.path.join(current_dir, '..'))

In [2]:
df_whole=pd.read_excel("E:\\科创优才\\实验数据\\天然气锅炉数据1.xlsx", sheet_name="稳定运行数据段")
units=df_whole.iloc[0].tolist()
df=df_whole.iloc[2:].reset_index(drop=True)

# 重命名所有列名，缩短名称长度，方便使用
df.columns=[
    '开始时间',
    '主蒸汽流量计算值',
    '烟气含氧量（CEMS）',
    '颗粒浓度',
    '二氧化硫',
    'NO浓度',
    'NO2浓度',
    'NOX标干浓度',
    'NOX浓度',
    '烟气湿度（CEMS）',
    '烟气压力（CEMS）',
    '烟气温度（CEMS）',
    '一氧化碳',
    '锅炉天然气进气流量',
    '锅炉天然气进气温度',
    '锅炉天然气进气压力',
    '鼓风机出口温度',
    '鼓风机出口压力',
    '鼓风机变频器输出反馈',
    '鼓风机变频器电流反馈',
    '再循环烟气调节阀反馈',
    '冷凝器出口烟气调节阀反馈',
    '炉膛出口烟气温度（B分度）',
    '炉膛出口烟气压力',
    'SWY大气压',
    'SWY天气温度',
    'SWY空气湿度',
    'SWY湿球温度',
    '主蒸汽温度(蒸汽集箱出口温度）',
    '主蒸汽压力(蒸汽集箱出口压力）',
    '分汽缸温度',
    '分汽缸压力',
    '分汽缸出口至DN400蒸汽温度',
    '过热器集箱出口蒸汽温度',
    '天然气累计流量',
    '冷凝器烟气流量（累计值）',
    '冷凝器出口烟气流量',
    '冷凝器出口烟气温度'
]

# 将可能用到的变量名和单位存入字典
var_dict={
    "主蒸汽流量计算值": "t/h",
    "烟气含氧量（CEMS）": "mg/Nm3",
    "NO浓度": "mg/Nm3",
    "NO2浓度": "mg/Nm3",
    "NOX浓度": "mg/Nm3",
    "烟气湿度（CEMS）": "%",
    "烟气压力（CEMS）": "Pa",
    "烟气温度（CEMS）": "℃",
    "一氧化碳": "mg/Nm3",
    "锅炉天然气进气流量": "m3/h",
    "锅炉天然气进气温度": "℃",
    "锅炉天然气进气压力": "kPa",
    '鼓风机出口温度': "℃",
    "鼓风机出口压力": "kPa",
    "鼓风机变频器输出反馈": "Hz",
    "鼓风机变频器电流反馈": "A",
    "冷凝器出口烟气调节阀反馈": "%",
    "炉膛出口烟气压力": "Pa",
    "SWY大气压": "kPa",
    "SWY天气温度": "℃",
    "SWY空气湿度": "%",
    'SWY湿球温度': "℃",
    '主蒸汽温度(蒸汽集箱出口温度）': "℃",
    "主蒸汽压力(蒸汽集箱出口压力）": "MPa",
    "分汽缸温度": "℃",
    "分汽缸压力": "MPa",
    '分汽缸出口至DN400蒸汽温度': "℃",
    '过热器集箱出口蒸汽温度': "℃",
    "冷凝器出口烟气流量": "Nm3/h",
    "冷凝器出口烟气温度": "℃",
}
var_names=list(var_dict.keys())

# 定义输入变量
input_var_names=[
    "主蒸汽流量计算值",
    "锅炉天然气进气流量",
    "锅炉天然气进气温度",
    "锅炉天然气进气压力",
    '鼓风机出口温度',
    "鼓风机出口压力",
    "鼓风机变频器输出反馈",
    "鼓风机变频器电流反馈",
    "冷凝器出口烟气调节阀反馈",
    "SWY大气压",
    "SWY天气温度",
    "SWY空气湿度",
    'SWY湿球温度',
    "主蒸汽温度(蒸汽集箱出口温度）",
    "主蒸汽压力(蒸汽集箱出口压力）",
]

# 定义输出变量
output_var_names=[
    "烟气含氧量（CEMS）",
    #NO浓度",
    #"NO2浓度", # 主要预测NO，因为NO2的准确性有待考量
    "NOX浓度",
    "烟气湿度（CEMS）",
    "烟气压力（CEMS）",
    "烟气温度（CEMS）",
    "一氧化碳",
    "炉膛出口烟气压力",

    #暂时不考虑以下输出变量
    #"分汽缸温度",
    #"分汽缸压力",
    #"分汽缸出口至DN400蒸汽温度",
    #"过热器集箱出口蒸汽温度",
    #"冷凝器出口烟气流量",
    #"冷凝器出口烟气温度",
]

input_var_dict={name:var_dict[name] for name in input_var_names}
output_var_dict={name:var_dict[name] for name in output_var_names}

var_units=list(var_dict.values())
input_var_units=list(input_var_dict.values())
output_var_units=list(output_var_dict.values())

input_var_indices=[var_names.index(name) for name in input_var_names]
output_var_indices=[var_names.index(name) for name in output_var_names]


data_np=df[var_names].to_numpy(dtype=float)

# 通过不同切片增加数据量
DATA=[
    data_np,
    data_np[:20,:],
    data_np[:50,:],
    data_np[:100,:],
    data_np[:800,:],
    data_np[100:850,:],
    data_np[200:900,:],
    data_np[300:950,:],
    data_np[400:1000,:],
    ]
print("data_np.shape:", data_np.shape)

data_np.shape: (1124, 30)


In [ ]:
class GasData():

    class Utils:
        def __init__(self, outer_instance):
            self.outer_instance=outer_instance

        @staticmethod
        def train_test_split(data, train_ratio=0.8, test_ratio=0.2):
            '''
            Split the dataset into training and testing sets.
            '''
            assert train_ratio+test_ratio<=1.0, "train_ratio+test_ratio must be <= 1.0"
            assert type(data)==list, "`data` must be a list"
            
            n_series=len(data)
            num_train=int(train_ratio*n_series)
            num_test=n_series-num_train

            # Randomly split the dataset into training and testing sets
            indices=list(range(n_series))
            import random
            random.shuffle(indices)
            train_series_indices=indices[:num_train]
            test_series_indices=indices[num_train:num_train+num_test]
            data_train = [data[i] for i in train_series_indices]
            data_test = [data[i] for i in test_series_indices]

            print("train data indices:", train_series_indices)
            print("test data indices:", test_series_indices)
            print("num_train:", num_train)
            print("num_test:", num_test)

            return (train_series_indices, test_series_indices), (data_train, data_test)

        @staticmethod
        def time_series_slice(data, input_len, output_len, input_indices=None, output_indices=None):
            r'''

            '''
            def split_2d_np(np_2d, input_len, output_len):
                assert np_2d.ndim==2, "`np_2d` must be a 2D numpy array"
                n_timesteps, n_vars = np_2d.shape
                X = []
                Y = []
                for i in range(0, n_timesteps-input_len-output_len+1, output_len):
                    X.append(np_2d[i:i+input_len,:])
                    Y.append(np_2d[i+input_len:i+input_len+output_len,:])
                X=np.array(X).astype("float32") # X shape: (N, input_len, n_vars)
                Y=np.array(Y).astype("float32") # Y shape: (N, output_len, n_vars)
                return X,Y

            assert type(data)==list, f"`data` should be a list, but got {type(data)}"
            assert all([isinstance(item, np.ndarray) for item in data]), "`data` should be a list of numpy arrays, but got non-numpy array in the list"
            assert all([item.ndim==2 for item in data]), "`data` must be a list of 2D numpy arrays, but got non-2D item in the list"
            assert all([item.shape[1]==data[0].shape[1] for item in data]), "All numpy arrays in `data` must have the same number of columns (features)"
            
            n_timesteps, n_vars = data[0].shape
            input_indices=range(n_vars) if input_indices is None else input_indices
            output_indices=range(n_vars) if output_indices is None else output_indices
            
            X_grouped=[]
            Y_grouped=[]
            for data_i in data: # data_i: numpy array. Shape: (n_timesteps, n_vars)
                X_i, Y_i = split_2d_np(data_i, input_len, output_len)
                X_i, Y_i = X_i[:,:,input_indices], Y_i[:,:,output_indices] # Only take the specified input and output variables into X and Y
                X_grouped.append(X_i) # X_i shape: (N, input_len, len(input_indices))
                Y_grouped.append(Y_i) # Y_i shape: (N, output_len, len(output_indices))

            return X_grouped, Y_grouped

    def __init__(self, data, input_len, output_len,
                    input_indices=None,
                    output_indices=None,
                    transform_func=None,
                    inverse_transform_func=None
                    ):
        r"""
        :param data: list of numpy arrays of shape: (n_timesteps, n_vars). n_timesteps can be different, but n_vars must be the same.
        :param input_len: the length of each input sequence.
        :param output_len: the length of each output sequence.
        :param input_indices: the indices of the input variables. If None, all variables are used as input variables.
        :param output_indices: the indices of the output variables. If None, all variables are used as output variables.
        """
        assert type(data)==list, "data must be a list"
        assert all([isinstance(item, np.ndarray) for item in data]) and all([item.ndim==2 for item in data]), "data must be a list of 2D numpy arrays"
        assert all([item.shape[1]==data[0].shape[1] for item in data]), "All numpy arrays in data must have the same number of columns (features)"

        self.frozen_data = data # save the original data, which is not changed during preprocessing
        self.data = data
        
        self.input_len = input_len
        self.output_len = output_len
        self.n_series = len(data)
        self.n_vars = data[0].shape[1]
        self.n_input_vars = self.n_vars if input_indices is None else len(input_indices)
        self.n_output_vars = self.n_vars if output_indices is None else len(output_indices)

        self.input_indices=range(self.n_vars) if input_indices is None else input_indices
        self.output_indices=range(self.n_vars) if output_indices is None else output_indices
        self.transform_func = lambda x: x if transform_func is None else transform_func
        self.inverse_transform_func = lambda x: x if transform_func is None else inverse_transform_func

        self.data_train = None # to be set by `self.train_test_split()`
        self.data_test = None # to be set by `self.train_test_split()`
        self.train_indices = None # to be set by `self.train_test_split()`
        self.test_indices = None # to be set by `self.train_test_split()`

        self.X_train_grouped = None # to be set by `self.time_series_slice()`
        self.Y_train_grouped = None # to be set by `self.time_series_slice()`
        self.X_test_grouped = None # to be set by `self.time_series_slice()`
        self.Y_test_grouped = None # to be set by `self.time_series_slice()`
        
        self.var_mean = None # to be set by `self.standardize()`
        self.var_std_dev = None # to be set by `self.standardize()`
        self.input_var_mean = None # to be set by `self.standardize()`
        self.input_var_std_dev = None # to be set by `self.standardize()`
        self.output_var_mean = None # to be set by `self.standardize()`
        self.output_var_std_dev = None # to be set by `self.standardize()`
        
        self.X_train = None # to be set by `self.build_train_test_set()`
        self.Y_train = None # to be set by `self.build_train_test_set()`
        self.X_test = None # to be set by `self.build_train_test_set()`
        self.Y_test = None # to be set by `self.build_train_test_set()`

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        return self.data[i]
    
    # --------The methods below should be called in order--------
    # --------If the order is changed, bugs may occur------------

    def transform(self):
        r'''
        Apply a transformation function to each numpy array in the data list.
        '''
        self.data = [self.transform_func(d) for d in self.data]
        return

    def standardize(self):
        count=np.sum([d.shape[0] for d in self.data]) # total timestep count
        var_mean = np.sum([np.sum(d, axis=0) for d in self.data], axis=0)/count
        var_std_dev=np.sqrt(np.sum([np.sum((d - var_mean) ** 2, axis=0) for d in self.data], axis=0)/count)
        transformed_data = [(d-var_mean)/var_std_dev for d in self.data]
        
        # Note that `self.data` is changed!
        self.data = transformed_data

        self.var_mean = var_mean
        self.var_std_dev = var_std_dev
        self.input_var_mean = var_mean[self.input_indices]
        self.input_var_std_dev = var_std_dev[self.input_indices]
        self.output_var_mean = var_mean[self.output_indices]
        self.output_var_std_dev = var_std_dev[self.output_indices]

        print("var_mean.shape: ", var_mean.shape)
        print("var_std_dev.shape: ", var_std_dev.shape)
        print("var_mean:", self.var_mean)
        print("var_std_dev:", self.var_std_dev)

        return transformed_data, var_mean, var_std_dev

    def train_test_split(self, train_ratio=0.8, test_ratio=0.2):
        (self.train_indices, self.test_indices), (self.data_train, self.data_test) \
            = self.Utils.train_test_split(self.data, train_ratio, test_ratio)

    def time_series_slice(self):
        r'''
        Segment the time series data into input and output sequences.
        Need to call `self.train_test_split()` first to split the data into training and testing sets.
        '''
        assert hasattr(self, "data_train") and \
            hasattr(self, "data_test"), \
            "Please call `self.train_test_split()` first to split the data into training and testing sets"
        
        self.X_train_grouped, self.Y_train_grouped = self.Utils.time_series_slice(self.data_train, self.input_len, self.output_len, self.input_indices, self.output_indices)
        self.X_test_grouped, self.Y_test_grouped = self.Utils.time_series_slice(self.data_test, self.input_len, self.output_len, self.input_indices, self.output_indices)
        
        print("len(X_train_grouped):", len(self.X_train_grouped))
        print("len(Y_train_grouped):", len(self.Y_train_grouped))
        print("len(X_test_grouped):", len(self.X_test_grouped))
        print("len(Y_test_grouped):", len(self.Y_test_grouped))

        return (self.X_train_grouped, self.Y_train_grouped), (self.X_test_grouped, self.Y_test_grouped)

    def build_train_test_set(self):
        r'''
        Get the training and testing sets (Organized as numpy arrays without a outer list) for time series data.
        Need to call `self.time_series_slice()` first to split the data into input and output sequences.
        '''
        assert hasattr(self, "X_train_grouped") and \
                hasattr(self, "Y_train_grouped") and \
                hasattr(self, "X_test_grouped") and \
                hasattr(self, "Y_test_grouped"), \
                "Please call `self.time_series_slice()` first to split the data into input and output sequences"
        
        self.X_train=np.concatenate(self.X_train_grouped, axis=0) # shape: (N_train, input_len, len(input_indices))
        self.Y_train=np.concatenate(self.Y_train_grouped, axis=0) # shape: (N_train, output_len, len(output_indices))
        self.X_test=np.concatenate(self.X_test_grouped, axis=0) # shape: (N_test, input_len, len(input_indices))
        self.Y_test=np.concatenate(self.Y_test_grouped, axis=0) # shape: (N_test, output_len, len(output_indices))
        
        print("X_train.shape:", self.X_train.shape)
        print("Y_train.shape:", self.Y_train.shape)
        print("X_test.shape:", self.X_test.shape)
        print("Y_test.shape:", self.Y_test.shape)
        
        return (self.X_train, self.Y_train), (self.X_test, self.Y_test)
    
    # -----------------------------------------
    def standardize_2d_np(self, np_2d, mode="input"):
        r'''
        Use the mean and standard deviation of the whole dataset to standardize a 2D numpy array.
        '''
        assert isinstance(np_2d, np.ndarray) and np_2d.ndim==2, "`np_2d` must be a 2D numpy array"
        assert self.var_mean is not None and self.var_std_dev is not None, \
                "Please call `self.standardize()` first to calculate the mean and standard deviation of the whole dataset"

        if mode=="input":
            assert np_2d.shape[1]==self.n_input_vars,\
                "The number of columns (features) of `np_2d` must be the same as the number of input variables in the dataset"            
            return (np_2d - self.input_var_mean) / self.input_var_std_dev
        elif mode=="output":
            assert np_2d.shape[1]==self.n_output_vars,\
                "The number of columns (features) of `np_2d` must be the same as the number of output variables in the dataset"
            return (np_2d - self.output_var_mean) / self.output_var_std_dev
        else:
            raise ValueError("`mode` must be either 'input' or 'output'")

    def inverse_standardize_2d_np(self, np_2d, mode="output"):
        r'''
        Use the mean and standard deviation of the whole dataset to inverse standardize a 2D numpy array.
        '''
        assert isinstance(np_2d, np.ndarray) and np_2d.ndim==2, "`np_2d` must be a 2D numpy array"
        assert self.var_mean is not None and self.var_std_dev is not None, \
                "Please call `self.standardize()` first to calculate the mean and standard deviation of the whole dataset"

        if mode=="output":
            assert np_2d.shape[1]==self.n_output_vars,\
                "The number of columns (features) of `np_2d` must be the same as the number of output variables in the dataset"
            return np_2d * self.output_var_std_dev + self.output_var_mean
        elif mode=="input":
            assert np_2d.shape[1]==self.n_input_vars,\
                "The number of columns (features) of `np_2d` must be the same as the number of input variables in the dataset"
            return np_2d * self.input_var_std_dev + self.input_var_mean
        else:
            raise ValueError("`mode` must be either 'input' or 'output'")
    
    def inverse_transform_2d_np(self, np_2d):
        r'''
        Apply the inverse transformation function to a 2D numpy array.
        '''
        return self.inverse_transform_func(np_2d)

In [115]:
DATASET=GasData(DATA, input_len=10, output_len=3, input_indices=input_var_indices, output_indices=output_var_indices)

In [116]:
DATASET.train_test_split()

train data indices: [4, 8, 6, 1, 7, 3, 2]
test data indices: [5, 0]
num_train: 7
num_test: 2


In [117]:
transformed_data, var_mean, var_std_dev = DATASET.standardize()

var_mean.shape:  (30,)
var_std_dev.shape:  (30,)
var_mean: [ 2.06925136e+01  5.29846683e+00  1.16397914e+01  1.62143930e+00
  1.32613705e+01  6.15553400e+00 -2.57731081e+01  4.91650563e+01
  2.49803087e+00  1.36146592e+03  1.32310847e+01  4.68488840e+01
  3.83799583e+01  4.59214643e+00  3.52757906e+01  1.54414176e+02
  1.00034873e+02  1.30232791e+00  9.55240718e+01  6.49332499e+00
  8.98026700e+01  5.62627034e+00  2.38506277e+02  8.01399666e-01
  2.54894501e+02  7.57025448e-01  2.62336099e+02  2.17039846e+02
  1.75798102e+01  4.69818356e+01]
var_std_dev: [5.25774915e+00 1.12384715e+00 2.77697843e+00 1.98819782e+00
 3.29911994e+00 8.07875049e-01 6.49194982e+00 2.70108226e+00
 1.43759549e+01 3.45937091e+02 1.02941032e+00 6.44667761e-01
 1.79798416e+00 6.67532835e-01 3.38456754e+00 3.13512112e+01
 1.24975791e-02 7.52194345e-01 2.01818040e-01 1.73927888e+00
 1.44895581e+01 9.53526721e-01 6.32014026e+00 4.23301983e-02
 6.34010834e+00 3.95721351e-02 6.39412758e+00 5.11072798e+00
 2.49742300e

In [94]:
(X_train_grouped, Y_train_grouped), (X_test_grouped, Y_test_grouped) = DATASET.time_series_slice()

len(X_train_grouped): 7
len(Y_train_grouped): 7
len(X_test_grouped): 2
len(Y_test_grouped): 2


In [95]:
(X_train, Y_train), (X_test, Y_test) = DATASET.build_train_test_set()

X_train.shape: (1306, 10, 15)
Y_train.shape: (1306, 3, 7)
X_test.shape: (259, 10, 15)
Y_test.shape: (259, 3, 7)
